In [27]:
import ast
import importlib
import os

import numpy as np
import pandas as pd
import torch
import torchaudio

import src.utils.AcousticParameterUtils
from src.utils import envelope as env

importlib.reload(src.utils.AcousticParameterUtils)

from src.utils.AcousticParameterUtils import (  # noqa: E402
    CenterTime,
    Clarity,
    Definition,
    EarlyDecayTime,
    ExtenededRIRModel,
    PercentageArticulationLoss,
    RapidSpeechTransmissionIndex,
    ReverberationTime,
    SchroederRIRModel,
    SparseStochasticIRModel,
)

In [2]:
rir_dataset_path = "./data/RIR_aggregated/RIR.data/"
rir_manifest_ID_resampled = pd.read_csv(
    "./data/RIR_aggregated/RIR.metadata/RIRLabelAugmentV2.csv"
)

In [38]:
mu = rir_manifest_ID_resampled["Th"].mean()
mu

0.039868226860636735

In [39]:
# randomly choose 100 RIRs to calculate the acoustic parameters to compare three RIR models
rir_manifest = rir_manifest_ID_resampled[:100]

# initialize calculators
sti_calculator = RapidSpeechTransmissionIndex()
alcons_calculator = PercentageArticulationLoss()
t60_calculator = ReverberationTime()
edt_calculator = EarlyDecayTime()
c80_calculator = Clarity(clarity_mode="C80")
c50_calculator = Clarity(clarity_mode="C50")
definition_calculator = Definition()
ts_calculator = CenterTime()

sti_gt = []
alcons_gt = []
t60_gt = []
edt_gt = []
c80_gt = []
c50_gt = []
d50_gt = []
ts_gt = []

sti_extended = []
alcons_extended = []
t60_extended = []
edt_extended = []
c80_extended = []
c50_extended = []
d50_extended = []
ts_extended = []

sti_sparse_stochastic = []
alcons_sparse_stochastic = []
t60_sparse_stochastic = []
edt_sparse_stochastic = []
c80_sparse_stochastic = []
c50_sparse_stochastic = []
d50_sparse_stochastic = []
ts_sparse_stochastic = []

# load the RIRs
for _, rir in enumerate(rir_manifest.iterrows()):
    rir = rir[1]
    rir_path = os.path.join(rir_dataset_path, rir["RIR"])
    rir_waveform, _ = torchaudio.load(rir_path)
    rir_waveform = rir_waveform.reshape(-1)

    # calculate the acoustic parameters (ground truth)
    sti = sti_calculator(rir_waveform)
    alcons = alcons_calculator(sti)
    t60 = t60_calculator(rir_waveform)
    edt = edt_calculator(rir_waveform)
    c80 = c80_calculator(rir_waveform)
    c50 = c50_calculator(rir_waveform)
    d50 = definition_calculator(rir_waveform)
    ts = ts_calculator(rir_waveform)

    sti_gt.append(sti)
    alcons_gt.append(alcons)
    t60_gt.append(t60)
    edt_gt.append(edt)
    c80_gt.append(c80)
    c50_gt.append(c50)
    d50_gt.append(d50)
    ts_gt.append(ts)

    Th, Tt = rir["Th"], rir["Tt"]

    # calculate the extended reverberation time
    extended_rir_model = ExtenededRIRModel(Th=Th, Tt=Tt, fs=16000)()
    extended_sti = sti_calculator(extended_rir_model)
    extended_alcons = alcons_calculator(extended_sti)
    extended_t60 = t60_calculator(extended_rir_model)
    extended_edt = edt_calculator(extended_rir_model)
    extended_c80 = c80_calculator(extended_rir_model)
    extended_c50 = c50_calculator(extended_rir_model)
    extended_d50 = definition_calculator(extended_rir_model)
    extended_ts = ts_calculator(extended_rir_model)

    sti_extended.append(extended_sti)
    alcons_extended.append(extended_alcons)
    t60_extended.append(extended_t60)
    edt_extended.append(extended_edt)
    c80_extended.append(extended_c80)
    c50_extended.append(extended_c50)
    d50_extended.append(extended_d50)
    ts_extended.append(extended_ts)

    # calculate the sparse stochastic reverberation time
    volume = (
        torch.tensor(ast.literal_eval(rir["volume"])).prod().round(decimals=0).item()
    )
    sparse_stochastic_rir_model = SparseStochasticIRModel(
        Ti=Th, Td=Tt, volume=volume, mu_Th=mu, fs=16000
    )()
    sparse_stochastic_sti = sti_calculator(sparse_stochastic_rir_model)
    sparse_stochastic_alcons = alcons_calculator(sparse_stochastic_sti)
    sparse_stochastic_t60 = t60_calculator(sparse_stochastic_rir_model)
    sparse_stochastic_edt = edt_calculator(sparse_stochastic_rir_model)
    sparse_stochastic_c80 = c80_calculator(sparse_stochastic_rir_model)
    sparse_stochastic_c50 = c50_calculator(sparse_stochastic_rir_model)
    sparse_stochastic_d50 = definition_calculator(sparse_stochastic_rir_model)
    sparse_stochastic_ts = ts_calculator(sparse_stochastic_rir_model)

    sti_sparse_stochastic.append(sparse_stochastic_sti)
    alcons_sparse_stochastic.append(sparse_stochastic_alcons)
    t60_sparse_stochastic.append(sparse_stochastic_t60)
    edt_sparse_stochastic.append(sparse_stochastic_edt)
    c80_sparse_stochastic.append(sparse_stochastic_c80)
    c50_sparse_stochastic.append(sparse_stochastic_c50)
    d50_sparse_stochastic.append(sparse_stochastic_d50)
    ts_sparse_stochastic.append(sparse_stochastic_ts)

In [42]:
# calculate the errors (MAE)
sti_error_extended = np.mean(np.abs(np.array(sti_gt) - np.array(sti_extended)))
sti_error_sparse_stochastic = np.mean(
    np.abs(np.array(sti_gt) - np.array(sti_sparse_stochastic))
)

alcons_error_extended = np.mean(np.abs(np.array(alcons_gt) - np.array(alcons_extended)))
alcons_error_sparse_stochastic = np.mean(
    np.abs(np.array(alcons_gt) - np.array(alcons_sparse_stochastic))
)

t60_error_extended = np.mean(np.abs(np.array(t60_gt) - np.array(t60_extended)))
t60_error_sparse_stochastic = np.mean(
    np.abs(np.array(t60_gt) - np.array(t60_sparse_stochastic))
)

edt_error_extended = np.mean(np.abs(np.array(edt_gt) - np.array(edt_extended)))
edt_error_sparse_stochastic = np.mean(
    np.abs(np.array(edt_gt) - np.array(edt_sparse_stochastic))
)

c80_error_extended = np.mean(np.abs(np.array(c80_gt) - np.array(c80_extended)))
c80_error_sparse_stochastic = np.mean(
    np.abs(np.array(c80_gt) - np.array(c80_sparse_stochastic))
)

c50_error_extended = np.mean(np.abs(np.array(c50_gt) - np.array(c50_extended)))
c50_error_sparse_stochastic = np.mean(
    np.abs(np.array(c50_gt) - np.array(c50_sparse_stochastic))
)

d50_error_extended = np.mean(np.abs(np.array(d50_gt) - np.array(d50_extended)))
d50_error_sparse_stochastic = np.mean(
    np.abs(np.array(d50_gt) - np.array(d50_sparse_stochastic))
)

ts_error_extended = np.mean(np.abs(np.array(ts_gt) - np.array(ts_extended)))
ts_error_sparse_stochastic = np.mean(
    np.abs(np.array(ts_gt) - np.array(ts_sparse_stochastic))
)

print(f"STI error (extended): {sti_error_extended}\n")
print(f"STI error (sparse stochastic): {sti_error_sparse_stochastic}\n")

print(f"ALCONS error (extended): {alcons_error_extended}\n")
print(f"ALCONS error (sparse stochastic): {alcons_error_sparse_stochastic}\n")

print(f"T60 error (extended): {t60_error_extended}\n")
print(f"T60 error (sparse stochastic): {t60_error_sparse_stochastic}\n")

print(f"EDT error (extended): {edt_error_extended}\n")
print(f"EDT error (sparse stochastic): {edt_error_sparse_stochastic}\n")

print(f"C80 error (extended): {c80_error_extended}\n")
print(f"C80 error (sparse stochastic): {c80_error_sparse_stochastic}\n")

print(f"C50 error (extended): {c50_error_extended}\n")
print(f"C50 error (sparse stochastic): {c50_error_sparse_stochastic}\n")

print(f"D50 error (extended): {d50_error_extended}\n")
print(f"D50 error (sparse stochastic): {d50_error_sparse_stochastic}\n")

print(f"TS error (extended): {ts_error_extended}\n")
print(f"TS error (sparse stochastic): {ts_error_sparse_stochastic}\n")

STI error (extended): 0.06863975524902344

STI error (sparse stochastic): 0.06850285083055496

ALCONS error (extended): 1.5056102275848389

ALCONS error (sparse stochastic): 1.5015605688095093

T60 error (extended): 0.07353999465703964

T60 error (sparse stochastic): 0.07354999333620071

EDT error (extended): 0.16787000000476837

EDT error (sparse stochastic): 0.16787999868392944

C80 error (extended): 1.6618067409330946

C80 error (sparse stochastic): 1.6611580848693848

C50 error (extended): 1.587097621420885

C50 error (sparse stochastic): 1.5862858295440674

D50 error (extended): 6.746828638341931

D50 error (sparse stochastic): 6.742861270904541

TS error (extended): 0.020753809548202065

TS error (sparse stochastic): 0.020750267431139946

